# Deep Learning Theory & Mathematical Foundations
## From Neurons to Attention: A Complete Guide

**Skill Levels Covered:** Beginner | Intermediate | Advanced | Architect

This notebook builds deep intuition for how neural networks work by implementing everything from scratch using NumPy before touching any framework. By the end, you'll understand not just *what* deep learning does, but *why* it works mathematically.

### Table of Contents
1. [The Neuron - From Biology to Math](#1)
2. [Neural Network Architecture](#2)
3. [Backpropagation - The Heart of Deep Learning](#3)
4. [Loss Functions Deep Dive](#4)
5. [Optimization Algorithms](#5)
6. [Regularization Techniques](#6)
7. [Convolution Operations Explained](#7)
8. [Feature Maps & Receptive Fields](#8)
9. [Weight Initialization Strategies](#9)
10. [The Mathematics of Attention](#10)
11. [Practical Exercises](#11)

<a name="1"></a>
## 1. The Neuron - From Biology to Math

### 1.1 Biological Inspiration

A biological neuron:
- Receives signals through **dendrites** (inputs)
- Processes them in the **cell body** (weighted sum + activation)
- Fires a signal through the **axon** (output) if threshold is exceeded

### 1.2 The Mathematical Model (Perceptron)

$$y = f\left(\sum_{i=1}^{n} w_i \cdot x_i + b\right) = f(\mathbf{w}^T \mathbf{x} + b)$$

Where:
- $\mathbf{x} = [x_1, x_2, ..., x_n]$ — input features
- $\mathbf{w} = [w_1, w_2, ..., w_n]$ — learnable weights
- $b$ — bias term
- $f$ — activation function (non-linearity)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# A single neuron implementation
class Neuron:
    def __init__(self, n_inputs):
        self.weights = np.random.randn(n_inputs) * 0.01
        self.bias = 0.0

    def forward(self, x, activation='relu'):
        z = np.dot(x, self.weights) + self.bias
        if activation == 'relu':
            return np.maximum(0, z)
        elif activation == 'sigmoid':
            return 1 / (1 + np.exp(-z))
        return z

# Demo: single neuron
neuron = Neuron(3)
x = np.array([1.0, 2.0, 3.0])
print(f"Input: {x}")
print(f"Weights: {neuron.weights}")
print(f"Bias: {neuron.bias}")
print(f"Output (ReLU): {neuron.forward(x, 'relu'):.4f}")
print(f"Output (Sigmoid): {neuron.forward(x, 'sigmoid'):.4f}")

### 1.3 Activation Functions

Activation functions introduce **non-linearity**, enabling neural networks to learn complex patterns. Without them, a network of any depth would collapse to a single linear transformation.

| Function | Formula | Range | Pros | Cons |
|----------|---------|-------|------|------|
| **Sigmoid** | $\sigma(x) = \frac{1}{1+e^{-x}}$ | (0, 1) | Smooth, probabilistic | Vanishing gradients, not zero-centered |
| **Tanh** | $\tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$ | (-1, 1) | Zero-centered | Vanishing gradients |
| **ReLU** | $\max(0, x)$ | [0, ∞) | Fast, sparse activation | Dead neurons |
| **Leaky ReLU** | $\max(0.01x, x)$ | (-∞, ∞) | No dead neurons | Small negative slope is arbitrary |
| **GELU** | $x \cdot \Phi(x)$ | (-∞, ∞) | Smooth ReLU, used in Transformers | Computationally expensive |
| **Swish** | $x \cdot \sigma(x)$ | (-∞, ∞) | Self-gated, smooth | Slightly expensive |

In [ ]:
# Activation functions visualization
def sigmoid(x): return 1 / (1 + np.exp(-x))
def relu(x): return np.maximum(0, x)
def tanh_fn(x): return np.tanh(x)
def leaky_relu(x): return np.where(x > 0, x, 0.01 * x)
def gelu(x): return 0.5 * x * (1 + np.tanh(np.sqrt(2/np.pi) * (x + 0.044715 * x**3)))
def swish(x): return x * sigmoid(x)

# Derivatives
def sigmoid_deriv(x): s = sigmoid(x); return s * (1 - s)
def relu_deriv(x): return np.where(x > 0, 1.0, 0.0)
def tanh_deriv(x): return 1 - np.tanh(x)**2

x = np.linspace(-5, 5, 1000)

fig, axes = plt.subplots(2, 3, figsize=(16, 9))
functions = [
    (sigmoid, sigmoid_deriv, 'Sigmoid', 'blue'),
    (relu, relu_deriv, 'ReLU', 'red'),
    (tanh_fn, tanh_deriv, 'Tanh', 'green'),
    (gelu, None, 'GELU', 'purple'),
    (swish, None, 'Swish', 'orange'),
    (leaky_relu, None, 'Leaky ReLU', 'brown')
]

for ax, (fn, deriv, name, color) in zip(axes.flat, functions):
    ax.plot(x, fn(x), linewidth=2.5, color=color, label=f'{name}(x)')
    if deriv is not None:
        ax.plot(x, deriv(x), linewidth=1.5, color=color, linestyle='--', alpha=0.6, label=f"{name}'(x)")
    ax.set_title(name, fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.axvline(x=0, color='k', linewidth=0.5)
    ax.legend(fontsize=9)
    ax.set_xlim(-5, 5)

plt.tight_layout()
plt.suptitle('Activation Functions & Their Derivatives', fontsize=16, fontweight='bold', y=1.02)
plt.show()

<a name="2"></a>
## 2. Neural Network Architecture

### 2.1 Layer Types

```
Input Layer    Hidden Layer(s)    Output Layer
   [x₁] ----\                 /---- [ŷ₁]
   [x₂] -----> [h₁] [h₂] [h₃] --> [ŷ₂]
   [x₃] ----/                 \---- [ŷ₃]
```

### 2.2 Forward Propagation

For a network with $L$ layers:

$$\mathbf{z}^{[l]} = \mathbf{W}^{[l]} \mathbf{a}^{[l-1]} + \mathbf{b}^{[l]}$$
$$\mathbf{a}^{[l]} = f^{[l]}(\mathbf{z}^{[l]})$$

where $\mathbf{a}^{[0]} = \mathbf{x}$ (the input).

In [ ]:
class NeuralNetwork:
    """A complete neural network built from scratch with NumPy."""

    def __init__(self, layer_sizes):
        self.num_layers = len(layer_sizes)
        self.params = {}
        self.cache = {}

        # He initialization for ReLU networks
        for l in range(1, self.num_layers):
            self.params[f'W{l}'] = np.random.randn(layer_sizes[l], layer_sizes[l-1]) * np.sqrt(2.0 / layer_sizes[l-1])
            self.params[f'b{l}'] = np.zeros((layer_sizes[l], 1))
            print(f"Layer {l}: W{l} shape = {self.params[f'W{l}'].shape}, b{l} shape = {self.params[f'b{l}'].shape}")

    def relu(self, Z):
        return np.maximum(0, Z)

    def softmax(self, Z):
        exp_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
        return exp_Z / exp_Z.sum(axis=0, keepdims=True)

    def forward(self, X):
        self.cache['A0'] = X
        A = X

        # Hidden layers use ReLU
        for l in range(1, self.num_layers - 1):
            Z = self.params[f'W{l}'] @ A + self.params[f'b{l}']
            A = self.relu(Z)
            self.cache[f'Z{l}'] = Z
            self.cache[f'A{l}'] = A

        # Output layer uses softmax
        l = self.num_layers - 1
        Z = self.params[f'W{l}'] @ A + self.params[f'b{l}']
        A = self.softmax(Z)
        self.cache[f'Z{l}'] = Z
        self.cache[f'A{l}'] = A

        return A

    def compute_loss(self, Y_pred, Y_true):
        m = Y_true.shape[1]
        loss = -np.sum(Y_true * np.log(Y_pred + 1e-8)) / m
        return loss

# Demo
nn = NeuralNetwork([784, 128, 64, 10])
X_dummy = np.random.randn(784, 5)  # 5 samples, 784 features
output = nn.forward(X_dummy)
print(f"\nInput shape: {X_dummy.shape}")
print(f"Output shape: {output.shape}")
print(f"Output sums to 1 (softmax check): {output.sum(axis=0)}")

<a name="3"></a>
## 3. Backpropagation - The Heart of Deep Learning

### 3.1 The Chain Rule

Backpropagation is simply the **chain rule of calculus** applied systematically through the network.

For a composition $f(g(h(x)))$:

$$\frac{df}{dx} = \frac{df}{dg} \cdot \frac{dg}{dh} \cdot \frac{dh}{dx}$$

### 3.2 Computational Graph

```
Forward Pass (left to right):
x ──→ [×W₁] ──→ z₁ ──→ [ReLU] ──→ a₁ ──→ [×W₂] ──→ z₂ ──→ [Softmax] ──→ ŷ ──→ [Loss] ──→ L

Backward Pass (right to left):
dL/dx ←── dL/dW₁ ←── dL/dz₁ ←── dL/da₁ ←── dL/dW₂ ←── dL/dz₂ ←── dL/dŷ ←── dL/dL = 1
```

### 3.3 Gradient Formulas

For the output layer:
$$\frac{\partial L}{\partial \mathbf{z}^{[L]}} = \hat{\mathbf{y}} - \mathbf{y}$$

For hidden layers:
$$\frac{\partial L}{\partial \mathbf{z}^{[l]}} = (\mathbf{W}^{[l+1]})^T \frac{\partial L}{\partial \mathbf{z}^{[l+1]}} \odot f'^{[l]}(\mathbf{z}^{[l]})$$

Parameter gradients:
$$\frac{\partial L}{\partial \mathbf{W}^{[l]}} = \frac{1}{m} \frac{\partial L}{\partial \mathbf{z}^{[l]}} (\mathbf{a}^{[l-1]})^T$$
$$\frac{\partial L}{\partial \mathbf{b}^{[l]}} = \frac{1}{m} \sum \frac{\partial L}{\partial \mathbf{z}^{[l]}}$$

In [ ]:
class TrainableNeuralNetwork(NeuralNetwork):
    """Neural network with backpropagation and training."""

    def relu_derivative(self, Z):
        return (Z > 0).astype(float)

    def backward(self, Y_true):
        m = Y_true.shape[1]
        grads = {}
        L = self.num_layers - 1

        # Output layer gradient (softmax + cross-entropy shortcut)
        dZ = self.cache[f'A{L}'] - Y_true

        for l in range(L, 0, -1):
            grads[f'dW{l}'] = (1/m) * dZ @ self.cache[f'A{l-1}'].T
            grads[f'db{l}'] = (1/m) * np.sum(dZ, axis=1, keepdims=True)

            if l > 1:
                dA = self.params[f'W{l}'].T @ dZ
                dZ = dA * self.relu_derivative(self.cache[f'Z{l-1}'])

        return grads

    def update_params(self, grads, learning_rate=0.01):
        for l in range(1, self.num_layers):
            self.params[f'W{l}'] -= learning_rate * grads[f'dW{l}']
            self.params[f'b{l}'] -= learning_rate * grads[f'db{l}']

    def train(self, X, Y, epochs=100, learning_rate=0.01, verbose=True):
        losses = []
        for epoch in range(epochs):
            # Forward
            Y_pred = self.forward(X)
            loss = self.compute_loss(Y_pred, Y)
            losses.append(loss)

            # Backward
            grads = self.backward(Y)
            self.update_params(grads, learning_rate)

            if verbose and epoch % 20 == 0:
                acc = np.mean(np.argmax(Y_pred, axis=0) == np.argmax(Y, axis=0))
                print(f"Epoch {epoch:4d} | Loss: {loss:.4f} | Accuracy: {acc:.4f}")

        return losses

# Train on synthetic spiral data
np.random.seed(42)
N = 100  # points per class
D = 2    # dimensions
K = 3    # classes

X_spiral = np.zeros((N*K, D))
Y_spiral = np.zeros(N*K, dtype=int)

for j in range(K):
    ix = range(N*j, N*(j+1))
    r = np.linspace(0.0, 1, N)
    t = np.linspace(j*4, (j+1)*4, N) + np.random.randn(N)*0.2
    X_spiral[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
    Y_spiral[ix] = j

# One-hot encode
Y_onehot = np.eye(K)[Y_spiral].T
X_T = X_spiral.T

# Train
nn = TrainableNeuralNetwork([2, 64, 32, 3])
losses = nn.train(X_T, Y_onehot, epochs=200, learning_rate=0.5)

# Plot results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curve
axes[0].plot(losses, linewidth=2, color='blue')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training Loss', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Decision boundary
h = 0.02
x_min, x_max = X_spiral[:, 0].min() - 0.5, X_spiral[:, 0].max() + 0.5
y_min, y_max = X_spiral[:, 1].min() - 0.5, X_spiral[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
grid = np.c_[xx.ravel(), yy.ravel()].T
probs = nn.forward(grid)
Z = np.argmax(probs, axis=0).reshape(xx.shape)

axes[1].contourf(xx, yy, Z, alpha=0.3, cmap='RdYlBu')
axes[1].scatter(X_spiral[:, 0], X_spiral[:, 1], c=Y_spiral, cmap='RdYlBu', edgecolors='black', s=30)
axes[1].set_title('Decision Boundary (Spiral Dataset)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3.4 Vanishing & Exploding Gradients

**Vanishing Gradients**: When using sigmoid/tanh, gradients shrink exponentially as they flow back through layers. Since $\sigma'(x) \leq 0.25$, after $n$ layers: $\prod_{i=1}^{n} 0.25 \to 0$

**Exploding Gradients**: When weights are large, gradients grow exponentially. Solutions include gradient clipping.

**Solutions:**
- Use **ReLU** (derivative is 0 or 1, no shrinkage)
- Use **skip connections** (ResNet)
- Use **batch normalization**
- Proper **weight initialization** (He, Xavier)

<a name="4"></a>
## 4. Loss Functions Deep Dive

### 4.1 Mean Squared Error (MSE)
$$L_{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

Best for: **Regression** problems

### 4.2 Binary Cross-Entropy
$$L_{BCE} = -\frac{1}{n} \sum_{i=1}^{n} [y_i \log(\hat{y}_i) + (1-y_i)\log(1-\hat{y}_i)]$$

Best for: **Binary classification**

### 4.3 Categorical Cross-Entropy
$$L_{CCE} = -\sum_{i=1}^{C} y_i \log(\hat{y}_i)$$

Best for: **Multi-class classification**

### 4.4 Focal Loss
$$L_{FL} = -\alpha_t (1-p_t)^\gamma \log(p_t)$$

Best for: **Imbalanced datasets** (reduces easy example contribution)

In [ ]:
# Loss functions implementation and visualization
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def binary_cross_entropy(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

def focal_loss(y_true, y_pred, gamma=2.0, alpha=0.25):
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
    focal_weight = (1 - p_t) ** gamma
    return -np.mean(alpha * focal_weight * np.log(p_t))

# Visualize loss landscapes
y_pred_range = np.linspace(0.01, 0.99, 200)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# MSE for y_true=1
axes[0].plot(y_pred_range, (1 - y_pred_range)**2, linewidth=2.5, color='blue', label='y_true=1')
axes[0].plot(y_pred_range, y_pred_range**2, linewidth=2.5, color='red', label='y_true=0')
axes[0].set_title('Mean Squared Error', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Predicted', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# BCE for y_true=1
axes[1].plot(y_pred_range, -np.log(y_pred_range), linewidth=2.5, color='blue', label='y_true=1')
axes[1].plot(y_pred_range, -np.log(1 - y_pred_range), linewidth=2.5, color='red', label='y_true=0')
axes[1].set_title('Binary Cross-Entropy', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Predicted', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].set_ylim(0, 5)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

# Focal loss for different gamma
for gamma in [0, 0.5, 1, 2, 5]:
    p_t = y_pred_range
    fl = -((1 - p_t) ** gamma) * np.log(p_t)
    axes[2].plot(y_pred_range, fl, linewidth=2, label=f'gamma={gamma}')
axes[2].set_title('Focal Loss (y_true=1)', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Predicted', fontsize=12)
axes[2].set_ylabel('Loss', fontsize=12)
axes[2].legend(fontsize=10)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

<a name="5"></a>
## 5. Optimization Algorithms

### 5.1 Gradient Descent Variants

**SGD (Stochastic Gradient Descent):**
$$\theta_{t+1} = \theta_t - \eta \nabla L(\theta_t)$$

**Momentum:**
$$v_t = \beta v_{t-1} + \eta \nabla L(\theta_t)$$
$$\theta_{t+1} = \theta_t - v_t$$

**RMSProp:**
$$s_t = \beta s_{t-1} + (1-\beta)(\nabla L)^2$$
$$\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{s_t + \epsilon}} \nabla L$$

**Adam (Adaptive Moment Estimation):**
$$m_t = \beta_1 m_{t-1} + (1-\beta_1) \nabla L$$
$$v_t = \beta_2 v_{t-1} + (1-\beta_2) (\nabla L)^2$$
$$\hat{m}_t = \frac{m_t}{1-\beta_1^t}, \quad \hat{v}_t = \frac{v_t}{1-\beta_2^t}$$
$$\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{\hat{v}_t} + \epsilon} \hat{m}_t$$

**AdamW (Adam with decoupled Weight Decay):**
Same as Adam, but adds $\lambda \theta_t$ directly to the update instead of through the gradient. This is the **modern default** optimizer.

In [ ]:
# Optimizer implementations and visualization on 2D loss surface
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
    def step(self, params, grads):
        return params - self.lr * grads

class Momentum:
    def __init__(self, lr=0.01, beta=0.9):
        self.lr, self.beta = lr, beta
        self.v = 0
    def step(self, params, grads):
        self.v = self.beta * self.v + self.lr * grads
        return params - self.v

class RMSProp:
    def __init__(self, lr=0.01, beta=0.99, eps=1e-8):
        self.lr, self.beta, self.eps = lr, beta, eps
        self.s = 0
    def step(self, params, grads):
        self.s = self.beta * self.s + (1 - self.beta) * grads**2
        return params - self.lr * grads / (np.sqrt(self.s) + self.eps)

class Adam:
    def __init__(self, lr=0.01, beta1=0.9, beta2=0.999, eps=1e-8):
        self.lr, self.beta1, self.beta2, self.eps = lr, beta1, beta2, eps
        self.m, self.v, self.t = 0, 0, 0
    def step(self, params, grads):
        self.t += 1
        self.m = self.beta1 * self.m + (1 - self.beta1) * grads
        self.v = self.beta2 * self.v + (1 - self.beta2) * grads**2
        m_hat = self.m / (1 - self.beta1**self.t)
        v_hat = self.v / (1 - self.beta2**self.t)
        return params - self.lr * m_hat / (np.sqrt(v_hat) + self.eps)

# Beale's function (challenging optimization surface)
def beale(x, y):
    return (1.5 - x + x*y)**2 + (2.25 - x + x*y**2)**2 + (2.625 - x + x*y**3)**2

def beale_grad(x, y):
    dx = 2*(1.5 - x + x*y)*(-1 + y) + 2*(2.25 - x + x*y**2)*(-1 + y**2) + 2*(2.625 - x + x*y**3)*(-1 + y**3)
    dy = 2*(1.5 - x + x*y)*x + 2*(2.25 - x + x*y**2)*(2*x*y) + 2*(2.625 - x + x*y**3)*(3*x*y**2)
    return np.array([dx, dy])

# Run optimizers
optimizers = {
    'SGD (lr=0.0001)': SGD(lr=0.0001),
    'Momentum (lr=0.0001)': Momentum(lr=0.0001, beta=0.9),
    'RMSProp (lr=0.005)': RMSProp(lr=0.005),
    'Adam (lr=0.01)': Adam(lr=0.01)
}

start = np.array([0.5, -1.5])
paths = {}

for name, opt in optimizers.items():
    pos = start.copy()
    path = [pos.copy()]
    for _ in range(300):
        grad = beale_grad(pos[0], pos[1])
        pos = opt.step(pos, grad)
        path.append(pos.copy())
    paths[name] = np.array(path)

# Plot
fig, ax = plt.subplots(figsize=(12, 9))
x_grid = np.linspace(-2, 4.5, 300)
y_grid = np.linspace(-2, 2.5, 300)
X, Y = np.meshgrid(x_grid, y_grid)
Z = beale(X, Y)

ax.contour(X, Y, Z, levels=np.logspace(-1, 3.5, 30), cmap='viridis', alpha=0.6)

colors = ['red', 'blue', 'green', 'orange']
for (name, path), color in zip(paths.items(), colors):
    ax.plot(path[:, 0], path[:, 1], '-o', color=color, markersize=2, linewidth=1.5, label=name)

ax.plot(3, 0.5, '*', color='gold', markersize=20, markeredgecolor='black', label='Global Minimum')
ax.set_title("Optimizer Trajectories on Beale's Function", fontsize=14, fontweight='bold')
ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.legend(fontsize=10, loc='upper left')
ax.set_xlim(-2, 4.5)
ax.set_ylim(-2, 2.5)
plt.tight_layout()
plt.show()

### 5.2 Learning Rate Schedules

| Schedule | Formula | Best For |
|----------|---------|----------|
| **Step Decay** | $\eta_t = \eta_0 \cdot \gamma^{\lfloor t/s \rfloor}$ | Simple training |
| **Cosine Annealing** | $\eta_t = \eta_{min} + \frac{1}{2}(\eta_{max}-\eta_{min})(1+\cos(\frac{t}{T}\pi))$ | Modern best practice |
| **Warmup + Cosine** | Linear warmup then cosine decay | Transformers, large models |
| **OneCycle** | Ramp up then ramp down | Fast convergence |

In [ ]:
# Learning rate schedule visualization
epochs = np.arange(100)

# Step decay
step_lr = 0.1 * (0.5 ** (epochs // 30))

# Cosine annealing
cosine_lr = 0.001 + 0.5 * (0.1 - 0.001) * (1 + np.cos(np.pi * epochs / 100))

# Warmup + cosine
warmup_epochs = 10
warmup_cosine = np.where(
    epochs < warmup_epochs,
    0.1 * epochs / warmup_epochs,
    0.001 + 0.5 * (0.1 - 0.001) * (1 + np.cos(np.pi * (epochs - warmup_epochs) / (100 - warmup_epochs)))
)

# Exponential decay
exp_lr = 0.1 * np.exp(-0.03 * epochs)

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(epochs, step_lr, linewidth=2, label='Step Decay')
ax.plot(epochs, cosine_lr, linewidth=2, label='Cosine Annealing')
ax.plot(epochs, warmup_cosine, linewidth=2, label='Warmup + Cosine')
ax.plot(epochs, exp_lr, linewidth=2, label='Exponential Decay')
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Learning Rate', fontsize=12)
ax.set_title('Learning Rate Schedules Comparison', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

<a name="6"></a>
## 6. Regularization Techniques

Regularization prevents **overfitting** — when a model memorizes training data instead of learning generalizable patterns.

### 6.1 L1 & L2 Regularization

**L2 (Ridge / Weight Decay):**
$$L_{total} = L_{data} + \frac{\lambda}{2m} \sum ||\mathbf{W}||^2_2$$

Effect: Pushes all weights toward zero, but doesn't make them exactly zero.

**L1 (Lasso):**
$$L_{total} = L_{data} + \frac{\lambda}{m} \sum ||\mathbf{W}||_1$$

Effect: Drives some weights to exactly zero → **sparse models**.

### 6.2 Dropout

During training, randomly set activations to zero with probability $p$:
$$\tilde{a}^{[l]} = \frac{\mathbf{m}^{[l]}}{1-p} \odot a^{[l]}$$

where $\mathbf{m}^{[l]}$ is a binary mask. The $\frac{1}{1-p}$ scaling (inverted dropout) keeps expected values consistent.

### 6.3 Batch Normalization

Normalize activations within a mini-batch:
$$\hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$$
$$y_i = \gamma \hat{x}_i + \beta$$

Where $\gamma$ (scale) and $\beta$ (shift) are **learnable parameters**.

In [ ]:
# Demonstrate overfitting vs regularization
np.random.seed(42)

# Generate noisy polynomial data
x_data = np.linspace(-3, 3, 30)
y_data = 0.5 * x_data**2 - x_data + np.random.randn(30) * 1.5

# Fit with different polynomial degrees
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for ax, degree, title in zip(axes, [2, 15, 15],
    ['Good Fit (degree=2)', 'Overfitting (degree=15)', 'Regularized (degree=15, L2)']):

    ax.scatter(x_data, y_data, color='blue', s=30, alpha=0.7, label='Data')

    if 'Regularized' in title:
        # Ridge regression (L2 regularized)
        X_poly = np.vander(x_data, degree + 1)
        lam = 10.0
        coeffs = np.linalg.solve(X_poly.T @ X_poly + lam * np.eye(degree + 1), X_poly.T @ y_data)
        x_plot = np.linspace(-3, 3, 200)
        X_plot = np.vander(x_plot, degree + 1)
        y_plot = X_plot @ coeffs
    else:
        coeffs = np.polyfit(x_data, y_data, degree)
        x_plot = np.linspace(-3, 3, 200)
        y_plot = np.polyval(coeffs, x_plot)

    ax.plot(x_plot, y_plot, color='red', linewidth=2, label='Model')
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.set_ylim(-10, 20)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

<a name="7"></a>
## 7. Convolution Operations Explained

### 7.1 What is a Convolution?

A convolution slides a **kernel** (filter) across an image, computing element-wise products and summing them at each position.

**Output size formula:**
$$O = \frac{I - K + 2P}{S} + 1$$

Where: $I$ = input size, $K$ = kernel size, $P$ = padding, $S$ = stride

### 7.2 Types of Convolutions

| Type | Description | Use Case |
|------|-------------|----------|
| **Standard** | Full KxK convolution | General feature extraction |
| **Depthwise** | Separate filter per channel | Efficient mobile models |
| **Pointwise (1x1)** | 1x1 convolution | Channel mixing / dimension change |
| **Depthwise Separable** | Depthwise + Pointwise | MobileNet, EfficientNet |
| **Dilated** | Gaps between kernel elements | Large receptive field without pooling |
| **Transposed** | "Upsampling" convolution | Decoder, segmentation |

In [ ]:
def conv2d_manual(image, kernel, stride=1, padding=0):
    """Manual 2D convolution implementation from scratch."""
    if padding > 0:
        image = np.pad(image, padding, mode='constant')

    h, w = image.shape
    kh, kw = kernel.shape
    out_h = (h - kh) // stride + 1
    out_w = (w - kw) // stride + 1
    output = np.zeros((out_h, out_w))

    for i in range(out_h):
        for j in range(out_w):
            region = image[i*stride:i*stride+kh, j*stride:j*stride+kw]
            output[i, j] = np.sum(region * kernel)
    return output

# Create sample image (checkerboard)
image = np.zeros((8, 8))
image[::2, ::2] = 1
image[1::2, 1::2] = 1

# Define common kernels
kernels = {
    'Edge Detection\n(Horizontal)': np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]]),
    'Edge Detection\n(Vertical)': np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]),
    'Sharpen': np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]),
    'Gaussian Blur': np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]]) / 16.0,
    'Sobel X': np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]),
    'Identity': np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]])
}

fig, axes = plt.subplots(2, len(kernels), figsize=(18, 7))

for idx, (name, kernel) in enumerate(kernels.items()):
    # Show kernel
    axes[0, idx].imshow(kernel, cmap='RdBu_r', vmin=-2, vmax=2)
    axes[0, idx].set_title(f'Kernel: {name}', fontsize=9, fontweight='bold')
    for i in range(kernel.shape[0]):
        for j in range(kernel.shape[1]):
            axes[0, idx].text(j, i, f'{kernel[i,j]:.1f}', ha='center', va='center', fontsize=8)
    axes[0, idx].set_xticks([]); axes[0, idx].set_yticks([])

    # Show result
    result = conv2d_manual(image, kernel, padding=1)
    axes[1, idx].imshow(result, cmap='gray')
    axes[1, idx].set_title('Output', fontsize=9)
    axes[1, idx].set_xticks([]); axes[1, idx].set_yticks([])

plt.suptitle('Convolution Kernels & Their Effects', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"Input size: {image.shape}")
print(f"Kernel size: 3x3")
print(f"Output size (no padding, stride=1): {conv2d_manual(image, kernels['Identity']).shape}")
print(f"Output size (padding=1, stride=1): {conv2d_manual(image, kernels['Identity'], padding=1).shape}")
print(f"Output size (padding=0, stride=2): {conv2d_manual(image, kernels['Identity'], stride=2).shape}")

<a name="8"></a>
## 8. Feature Maps & Receptive Fields

### 8.1 Receptive Field

The **receptive field** is the region of the input image that affects a particular feature in a deeper layer.

**Formula for receptive field size:**
$$r_l = r_{l-1} + (k_l - 1) \times \prod_{i=1}^{l-1} s_i$$

Where $r_l$ = receptive field at layer $l$, $k_l$ = kernel size, $s_i$ = stride at layer $i$.

### 8.2 Why Receptive Fields Matter

| Layer Depth | Receptive Field | What It "Sees" |
|-------------|----------------|----------------|
| Early (1-2) | Small (3-7px) | Edges, textures |
| Middle (3-5) | Medium (20-50px) | Parts, patterns |
| Deep (6+) | Large (100px+) | Objects, scenes |

In [ ]:
def calculate_receptive_field(layers):
    """Calculate receptive field for a sequence of conv/pool layers.
    Each layer: (kernel_size, stride, padding)
    """
    r = 1  # receptive field
    j = 1  # jump (product of strides)

    print(f"{'Layer':<20} {'Kernel':<8} {'Stride':<8} {'RF Size':<10} {'Jump':<8}")
    print("-" * 60)
    print(f"{'Input':<20} {'-':<8} {'-':<8} {r:<10} {j:<8}")

    for name, (k, s, p) in layers:
        r = r + (k - 1) * j
        j = j * s
        print(f"{name:<20} {k:<8} {s:<8} {r:<10} {j:<8}")

    return r

# VGG-16 receptive field
print("=== VGG-16 Receptive Field ===")
vgg_layers = [
    ('Conv3x3_1', (3, 1, 1)), ('Conv3x3_2', (3, 1, 1)),
    ('MaxPool', (2, 2, 0)),
    ('Conv3x3_3', (3, 1, 1)), ('Conv3x3_4', (3, 1, 1)),
    ('MaxPool', (2, 2, 0)),
    ('Conv3x3_5', (3, 1, 1)), ('Conv3x3_6', (3, 1, 1)), ('Conv3x3_7', (3, 1, 1)),
    ('MaxPool', (2, 2, 0)),
    ('Conv3x3_8', (3, 1, 1)), ('Conv3x3_9', (3, 1, 1)), ('Conv3x3_10', (3, 1, 1)),
    ('MaxPool', (2, 2, 0)),
    ('Conv3x3_11', (3, 1, 1)), ('Conv3x3_12', (3, 1, 1)), ('Conv3x3_13', (3, 1, 1)),
    ('MaxPool', (2, 2, 0)),
]
rf = calculate_receptive_field(vgg_layers)
print(f"\nFinal receptive field of VGG-16: {rf}x{rf} pixels")

print("\n=== ResNet-50 (first block) Receptive Field ===")
resnet_layers = [
    ('Conv7x7/2', (7, 2, 3)),
    ('MaxPool3x3/2', (3, 2, 1)),
    ('Conv1x1', (1, 1, 0)), ('Conv3x3', (3, 1, 1)), ('Conv1x1', (1, 1, 0)),
]
rf = calculate_receptive_field(resnet_layers)
print(f"\nReceptive field after first ResNet block: {rf}x{rf} pixels")

<a name="9"></a>
## 9. Weight Initialization Strategies

### Why Initialization Matters

Bad initialization → vanishing or exploding activations → poor training.

| Method | Formula | Best With |
|--------|---------|-----------|
| **Xavier/Glorot** | $W \sim \mathcal{N}(0, \frac{2}{n_{in}+n_{out}})$ | Sigmoid, Tanh |
| **He** | $W \sim \mathcal{N}(0, \frac{2}{n_{in}})$ | ReLU family |
| **LeCun** | $W \sim \mathcal{N}(0, \frac{1}{n_{in}})$ | SELU |

### The Key Principle

Keep the **variance of activations** roughly constant across layers. If variance grows → exploding; if variance shrinks → vanishing.

In [ ]:
# Experiment: Effect of initialization on activation distributions
np.random.seed(42)

layer_sizes = [784] + [256] * 10  # 10 hidden layers of size 256

init_methods = {
    'Random (std=1.0)': lambda fan_in, fan_out: np.random.randn(fan_in, fan_out) * 1.0,
    'Random (std=0.01)': lambda fan_in, fan_out: np.random.randn(fan_in, fan_out) * 0.01,
    'Xavier': lambda fan_in, fan_out: np.random.randn(fan_in, fan_out) * np.sqrt(2.0 / (fan_in + fan_out)),
    'He': lambda fan_in, fan_out: np.random.randn(fan_in, fan_out) * np.sqrt(2.0 / fan_in),
}

fig, axes = plt.subplots(len(init_methods), len(layer_sizes)-1, figsize=(20, 10))

for row, (name, init_fn) in enumerate(init_methods.items()):
    x = np.random.randn(200, 784)  # batch of 200

    for col in range(len(layer_sizes) - 1):
        W = init_fn(layer_sizes[col], layer_sizes[col + 1])
        x = np.maximum(0, x @ W)  # ReLU

        axes[row, col].hist(x.flatten(), bins=50, density=True, alpha=0.7, color='steelblue')
        axes[row, col].set_xlim(-1, 5)

        if col == 0:
            axes[row, col].set_ylabel(name, fontsize=10, fontweight='bold')
        if row == 0:
            axes[row, col].set_title(f'Layer {col+1}', fontsize=10)

        mean_val = x.mean()
        std_val = x.std()
        axes[row, col].text(0.7, 0.9, f'μ={mean_val:.2f}\nσ={std_val:.2f}',
                           transform=axes[row, col].transAxes, fontsize=7,
                           verticalalignment='top')

plt.suptitle('Activation Distributions Across Layers (ReLU)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

<a name="10"></a>
## 10. The Mathematics of Attention

### 10.1 Self-Attention Mechanism

Attention allows the model to focus on different parts of the input for each output element.

**Scaled Dot-Product Attention:**
$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

Where:
- $Q$ (Query): "What am I looking for?"
- $K$ (Key): "What do I contain?"
- $V$ (Value): "What information do I provide?"
- $d_k$: dimension of keys (scaling factor prevents large dot products)

### 10.2 Multi-Head Attention

$$\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, ..., \text{head}_h)W^O$$
$$\text{where head}_i = \text{Attention}(QW^Q_i, KW^K_i, VW^V_i)$$

This allows the model to attend to different representation subspaces simultaneously.

In [ ]:
def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    Scaled Dot-Product Attention.

    Args:
        Q: Queries [batch, heads, seq_len, d_k]
        K: Keys    [batch, heads, seq_len, d_k]
        V: Values  [batch, heads, seq_len, d_v]
    """
    d_k = Q.shape[-1]

    # Compute attention scores
    scores = Q @ K.transpose(0, 1, 3, 2) / np.sqrt(d_k)

    if mask is not None:
        scores = np.where(mask, scores, -1e9)

    # Softmax
    exp_scores = np.exp(scores - np.max(scores, axis=-1, keepdims=True))
    attention_weights = exp_scores / exp_scores.sum(axis=-1, keepdims=True)

    # Weighted sum of values
    output = attention_weights @ V

    return output, attention_weights

class MultiHeadAttention:
    def __init__(self, d_model, num_heads):
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        # Initialize projection matrices
        self.W_Q = np.random.randn(d_model, d_model) * 0.02
        self.W_K = np.random.randn(d_model, d_model) * 0.02
        self.W_V = np.random.randn(d_model, d_model) * 0.02
        self.W_O = np.random.randn(d_model, d_model) * 0.02

    def split_heads(self, x):
        batch, seq_len, d_model = x.shape
        return x.reshape(batch, seq_len, self.num_heads, self.d_k).transpose(0, 2, 1, 3)

    def forward(self, x):
        batch, seq_len, d_model = x.shape

        Q = self.split_heads(x @ self.W_Q)
        K = self.split_heads(x @ self.W_K)
        V = self.split_heads(x @ self.W_V)

        attn_output, attn_weights = scaled_dot_product_attention(Q, K, V)

        # Concatenate heads
        attn_output = attn_output.transpose(0, 2, 1, 3).reshape(batch, seq_len, d_model)
        output = attn_output @ self.W_O

        return output, attn_weights

# Demo
np.random.seed(42)
seq_len, d_model, num_heads = 6, 64, 8

# Simulate a sequence of 6 tokens with 64-dim embeddings
x = np.random.randn(1, seq_len, d_model)
mha = MultiHeadAttention(d_model, num_heads)
output, attn_weights = mha.forward(x)

print(f"Input shape:  {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Attention weights shape: {attn_weights.shape}")

# Visualize attention weights for each head
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
tokens = ['Token_0', 'Token_1', 'Token_2', 'Token_3', 'Token_4', 'Token_5']

for i, ax in enumerate(axes.flat):
    im = ax.imshow(attn_weights[0, i], cmap='Blues', vmin=0, vmax=0.5)
    ax.set_title(f'Head {i+1}', fontsize=12, fontweight='bold')
    ax.set_xticks(range(seq_len))
    ax.set_yticks(range(seq_len))
    ax.set_xticklabels(tokens, rotation=45, fontsize=8)
    ax.set_yticklabels(tokens, fontsize=8)

plt.suptitle('Multi-Head Attention Weights (8 Heads)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

<a name="11"></a>
## 11. Practical Exercises

### Exercise 1: XOR Problem (Beginner)
Implement a 2-layer neural network from scratch that solves the XOR problem. XOR is not linearly separable, proving the need for hidden layers.

### Exercise 2: Gradient Checking (Intermediate)
Implement numerical gradient checking to verify your backpropagation:
$$\frac{\partial L}{\partial \theta} \approx \frac{L(\theta + \epsilon) - L(\theta - \epsilon)}{2\epsilon}$$

### Exercise 3: Custom Optimizer (Advanced)
Implement the AdamW optimizer from scratch and compare it with Adam on a training task.

### Exercise 4: Architecture Design (Architect)
Design a neural network architecture for a specific constraint:
- Input: 28x28 grayscale images
- Maximum 50K parameters
- Target: >95% accuracy on MNIST
- Must include: Conv layers, BatchNorm, residual connection

In [ ]:
# Exercise 1: XOR Problem from Scratch
print("=" * 50)
print("Exercise 1: Solving XOR with a Neural Network")
print("=" * 50)

# XOR data
X_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).T  # (2, 4)
Y_xor = np.array([[0, 1, 1, 0]])  # (1, 4)

# Small network: 2 -> 4 -> 1
np.random.seed(42)
W1 = np.random.randn(4, 2) * 0.5
b1 = np.zeros((4, 1))
W2 = np.random.randn(1, 4) * 0.5
b2 = np.zeros((1, 1))

lr = 1.0
losses = []

for epoch in range(10000):
    # Forward
    Z1 = W1 @ X_xor + b1
    A1 = np.maximum(0, Z1)  # ReLU
    Z2 = W2 @ A1 + b2
    A2 = 1 / (1 + np.exp(-Z2))  # Sigmoid

    # Loss
    loss = -np.mean(Y_xor * np.log(A2 + 1e-8) + (1 - Y_xor) * np.log(1 - A2 + 1e-8))
    losses.append(loss)

    # Backward
    dZ2 = A2 - Y_xor
    dW2 = (1/4) * dZ2 @ A1.T
    db2 = (1/4) * np.sum(dZ2, axis=1, keepdims=True)
    dA1 = W2.T @ dZ2
    dZ1 = dA1 * (Z1 > 0)
    dW1 = (1/4) * dZ1 @ X_xor.T
    db1 = (1/4) * np.sum(dZ1, axis=1, keepdims=True)

    # Update
    W1 -= lr * dW1; b1 -= lr * db1
    W2 -= lr * dW2; b2 -= lr * db2

# Results
predictions = (A2 > 0.5).astype(int)
print(f"\nInput -> Expected -> Predicted")
for i in range(4):
    print(f"  {X_xor[:, i]} ->    {Y_xor[0, i]}     ->    {predictions[0, i]}  (prob: {A2[0, i]:.4f})")

print(f"\nFinal loss: {losses[-1]:.6f}")
print(f"All correct: {np.all(predictions == Y_xor)}")

# Plot loss
plt.figure(figsize=(10, 4))
plt.plot(losses, linewidth=2, color='blue')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('XOR Training Loss (From Scratch)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Summary

| Concept | Key Takeaway |
|---------|-------------|
| **Neuron** | $y = f(\mathbf{w}^T\mathbf{x} + b)$ — weighted sum + non-linearity |
| **Backpropagation** | Chain rule applied through computational graph |
| **Loss Functions** | Cross-entropy for classification, MSE for regression |
| **Optimizers** | Adam/AdamW are the modern defaults |
| **Regularization** | Dropout + BatchNorm + weight decay = standard recipe |
| **Convolutions** | Local feature extraction with weight sharing |
| **Initialization** | He init for ReLU, Xavier for sigmoid/tanh |
| **Attention** | $\text{softmax}(QK^T/\sqrt{d_k})V$ — the foundation of Transformers |

**Next Notebook:** [Deep Learning Pet Classifier Project](DeepLearningProject_Assignment.ipynb) — Apply these concepts to build real models!